Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## 02. Register Model, Create Image and Deploy Service

This example shows how to deploy a web service in step-by-step fashion:

 1. Register model
 2. Query versions of models and select one to deploy
 3. Create Docker image
 4. Query versions of images
 5. Deploy the image as web service
  
The train-within-notebook example taught you how to deploy a web service directly from model in one step. This Notebook shows a more advanced approach that gives you more control over model versions and Docker image versions.  

## Prerequisites
Make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### Register Model

Let's say we found pi as 3.1415926535

In [ ]:
import pickle

with open(".\pi.txt","wb") as f:
    pickle.dump(str('3.1415926535'),f)

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = "./pi.txt",
                       model_name = "Pi_3.14",
                       tags = {'area': "calc", 'type': "aml101", 'version': 1.0},
                       description = "Azure Machine Learning Service Concepts",
                       workspace = ws)

You can explore the registered models within your workspace and query by tag. Models are versioned. If you call the register_model command many times with same model name, you will get multiple versions of the model with increasing version numbers.

In [ ]:
models = Model.list(workspace=ws, tags=['area'])
for m in models:
    print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

You can pick a specific model to deploy

In [ ]:
print(model.name, model.description, model.version, sep = '\t')

### Create Docker Image

Show `score.py`. Note that the `sklearn_regression_model.pkl` in the `get_model_path` call is referring to a model named `sklearn_linreg_model.pkl` registered under the workspace. It is NOT referenceing the local file.

In [ ]:
%%writefile score.py
import pickle, json
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path(model_name = 'Pi_3.14')
    with open(model_path, "rb") as f:
        model = float(pickle.load(f))

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        radius = json.loads(raw_data)["radius"]
        result = model * radius**2
        return json.dumps({"area": result})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

cd = CondaDependencies()
cd.save_to_file(".", "myenv.yml")

Note that following command can take few minutes. 

You can add tags and descriptions to images. Also, an image can contain multiple models.

In [ ]:
from azureml.core.image import Image, ContainerImage

# Specify the configuration of image: scoring script, Python runtime (PySpark is the other option), and conda file of library dependencies.
image_config = ContainerImage.image_configuration(execution_script = "score.py", 
                                    runtime = "python", 
                                    tags = {'area':'calc','type':'aml101'},
                                    conda_file = "myenv.yml")

image = Image.create(name = "area-calc-img",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

In [ ]:
image.wait_for_creation(show_output = True)

List images by tag and find out the detailed build log for debugging.

In [ ]:
for i in Image.list(workspace = ws,tags = ["area"]):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

### Deploy image as web service on Azure Container Instance

Note that the service creation can take few minutes.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "calc", 'type': "aml101"}, 
                                               description = 'Calculate circle area using pi 3.141592')

In [ ]:
from azureml.core.webservice import Webservice

aci_service_name = 'area-web-calculator'
print(aci_service_name)

service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
service.wait_for_deployment(True)
print(aci_service.state)

### Test web service

Call the web service with some dummy input data to get a prediction.

In [ ]:
from ipywidgets import interact
import json

def get_area(radius):
    request = json.dumps({"radius": radius})
    response = service.run(input_data = request)
    return json.loads(response)["area"]

interact(get_area,radius=(0,10))

### Delete ACI to clean up

In [ ]:
service.delete()